# Create Table

In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists
from util.config import setting
from db.service.database import engine
from db.service.model import Base

if database_exists(setting.testdb_uri):
    print("testdb already exists")
if not database_exists(setting.testdb_uri):
    create_database(setting.testdb_uri)
Base.metadata.create_all(bind=engine)
print("testdb created")

testdb created


# Insert value into table

In [1]:
from db.service.model import Student
from db.service.database import SessionLocal
from db.crud.insert import general_insert_value

row_list = [{"name":"Lisa", "age":20},
            {"name":"Charlie", "age":21},
            {"name":"David", "age":20},
            {"name":"Adam", "age":21},
            {"name":"Chris", "age":24},
            {"name":"Bobby", "age":23},]

for row in row_list:
    general_insert_value(SessionLocal, Student, row)

# Read data form table

In [2]:
with SessionLocal() as session:
    query_results = session.query(Student).all()

print(query_results)

[<db.service.model.Student object at 0x109e08130>, <db.service.model.Student object at 0x109e080a0>, <db.service.model.Student object at 0x109e08160>, <db.service.model.Student object at 0x109e081c0>, <db.service.model.Student object at 0x109e08220>, <db.service.model.Student object at 0x109e08250>]


In [3]:
from db.util import query_results_to_dict
query_results_to_dict(query_results)

[{'age': 20, 'name': 'Lisa', 'id': 3},
 {'age': 21, 'name': 'Charlie', 'id': 4},
 {'age': 20, 'name': 'David', 'id': 5},
 {'age': 21, 'name': 'Adam', 'id': 6},
 {'age': 24, 'name': 'Chris', 'id': 7},
 {'age': 23, 'name': 'Bobby', 'id': 8}]

In [4]:
with SessionLocal() as session:
    query_results = session.query(Student.name, Student.age).all()

print(query_results)

[('Lisa', 20), ('Charlie', 21), ('David', 20), ('Adam', 21), ('Chris', 24), ('Bobby', 23)]


In [5]:
query_results_to_dict(query_results)

[{'name': 'Lisa', 'age': 20},
 {'name': 'Charlie', 'age': 21},
 {'name': 'David', 'age': 20},
 {'name': 'Adam', 'age': 21},
 {'name': 'Chris', 'age': 24},
 {'name': 'Bobby', 'age': 23}]

# Read data from table using SingleTableSQLInterface

In [6]:
from db.crud.read import SingleTableSQLInterface
columns = ["name", "age"]
orm_class = Student

single_table_session = SingleTableSQLInterface(columns, SessionLocal, orm_class)

required_dict = {'equal_condition' : {'age':20},
                'order_by_condition':{'column':'age', 'option':"asc"},
                # 'between_condition': {'column': 'signal_noise_ratio', 'from':10, 'end':11},
                'limit_condition':100}

single_table_session.load_query_result(**required_dict)


[{'name': 'Lisa', 'age': 20}, {'name': 'David', 'age': 20}]

In [7]:
columns = ["name", "age"]
orm_class = Student

single_table_session = SingleTableSQLInterface(columns, SessionLocal, orm_class)

required_dict = {'order_by_condition':{'column':'age', 'option':"desc"},
                'between_condition': {'column': 'age', 'from':10, 'end':31},
                'limit_condition':100}

single_table_session.load_query_result(**required_dict)

[{'name': 'Chris', 'age': 24},
 {'name': 'Bobby', 'age': 23},
 {'name': 'Charlie', 'age': 21},
 {'name': 'Adam', 'age': 21},
 {'name': 'Lisa', 'age': 20},
 {'name': 'David', 'age': 20}]

In [8]:
columns = ["name", "age"]
orm_class = Student

single_table_session = SingleTableSQLInterface(columns, SessionLocal, orm_class)

required_dict = {'order_by_condition':{'column':'age', 'option':"desc"},
                'between_condition': {'column': 'age', 'from':10, 'end':31},
                'limit_condition':2}

single_table_session.load_query_result(**required_dict)

[{'name': 'Chris', 'age': 24}, {'name': 'Bobby', 'age': 23}]